In [4]:
import pandas as pd
import numpy as np

d1 = pd.read_csv("data/材料1.csv")
d2 = pd.read_csv("data/材料2.csv")
d3 = pd.read_csv("data/材料3.csv")
d4 = pd.read_csv("data/材料4.csv")

d1['材料'] = '材料1'
d2['材料'] = '材料2'
d3['材料'] = '材料3'
d4['材料'] = '材料4'

data = pd.concat([d1, d2, d3, d4])


magnetic_flux_density = data.iloc[:, 4:-1].values

std_flux = np.std(magnetic_flux_density, axis=1)  # 标准差
max_flux = np.max(magnetic_flux_density, axis=1)  # 峰值

data['标准差'] = std_flux
data['峰值'] = max_flux

lst = ['温度', '频率', "磁芯损耗", '励磁波形']
for i in range(1024):
    lst.append(i)
lst.append('使用材料')
lst.append('标准差')
lst.append('峰值')
data.columns = lst



filter_data = data[['温度', '频率', '励磁波形', '使用材料', '标准差', '峰值', '磁芯损耗']]

print(filter_data.head())

   温度     频率 励磁波形 使用材料       标准差        峰值         磁芯损耗
0  25  50030  正弦波  材料1  0.020400  0.028849  1997.955250
1  25  50020  正弦波  材料1  0.022223  0.031419  2427.749830
2  25  50020  正弦波  材料1  0.025107  0.035535  3332.725760
3  25  50020  正弦波  材料1  0.028263  0.040015  4502.908007
4  25  50030  正弦波  材料1  0.031812  0.045028  6063.023248


In [5]:
from sklearn.preprocessing import LabelEncoder

""" 因果森林
"""
import numpy as np
from sklearn.model_selection import train_test_split
from econml.dml import CausalForestDML
from sklearn.ensemble import RandomForestRegressor

# 第一步：加载已经合并并清理好的数据
data = filter_data.copy()

# 删除包含 NaN 值的行
data = data.dropna()

# 对励磁波形和材料进行标签编码
label_encoder_waveform = LabelEncoder()
label_encoder_material = LabelEncoder()

data['励磁波形'] = label_encoder_waveform.fit_transform(data['励磁波形'])
data['材料'] = label_encoder_material.fit_transform(data['使用材料'])

lst_t = ['励磁波形', '材料', '温度']
X = data[['标准差', '峰值']]  # 自变量
T = data[lst_t]
y = data['磁芯损耗']  # 因变量

# 第三步：将数据分为训练集和测试集
X_train, X_test, T_train, T_test, y_train, y_test = train_test_split(X, T, y, test_size=0.2, random_state=42)

y_test = y_test.ravel()
y_train = y_train.ravel()

# 定义因果森林模型
# 使用随机森林回归器作为模型来拟合处理变量 (T) 和结果变量 (Y)
model_t = RandomForestRegressor(n_estimators=100, min_samples_leaf=5)
model_y = RandomForestRegressor(n_estimators=100, min_samples_leaf=5)

# 初始化因果森林模型
causal_forest = CausalForestDML(model_y=model_y, model_t=model_t, discrete_treatment=False)

# 将温度作为处理变量 (T)，X_train 作为协变量，y_train 作为结果
# T_train = X_train['温度']  # 选择一个作为处理变量
# X_train_new = X_train[['励磁波形', '材料', '标准差', '峰值']]  # 其他特征作为协变量

# 训练因果森林模型
causal_forest.fit(Y=y_train, T=T_train, X=X_train)

# 第三步：估计处理效应
# 对测试集进行个体处理效应估计

treatment_effects = causal_forest.effect(X_test)

# y_pred = causal_forest.model_y_.predict(X_test)


# 输出个体化处理效应
s = ""
for i in lst_t:
    s += i

# 计算均方误差（MSE）
# mse = mean_squared_error(y_test, y_pred)
# print(f"{s} 均方误差 (MSE): {mse}")

"""
处理效应（Treatment Effect） 是在因果推断中用于量化某种干预（或处理）对结果的影响。处理效应可以理解为：在其他条件相同的情况下，某个处理或干预的引入（例如温度的升高、使用不同的材料、采取某种政策等）对目标结果（例如磁芯损耗、经济增长、治疗效果等）造成的影响。

"""

print(f"{s} 对磁芯损耗的处理效应:")
print(treatment_effects)
# 

df = pd.DataFrame(treatment_effects)
df.to_excel(f"~/Desktop/{s}_对磁芯损耗的处理效应.xlsx")


IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.


励磁波形材料温度 对磁芯损耗的处理效应:
[   2010.71212185  -90705.51527831 -167840.89604005 ... -235773.80976249
 -145871.96669985 -373071.92964046]


A scalar was specified but there are multiple treatments; the same value will be used for each treatment.  Consider specifyingall treatments, or using the const_marginal_effect method.


## XGBOOST 回归

In [6]:
# from matplotlib import pyplot as plt
# import matplotlib.font_manager as fm
# import xgboost as xgb
# from sklearn.metrics import mean_squared_error
# from sklearn.model_selection import train_test_split
# from sklearn.inspection import PartialDependenceDisplay
# 
# 
# 
# label_encoder_waveform = LabelEncoder()
# label_encoder_material = LabelEncoder()
# 
# data['波形_encoded'] = label_encoder_waveform.fit_transform(data['波形'])
# data['材料_encoded'] = label_encoder_material.fit_transform(data['材料'])
# 
# X = data[['温度', '波形_encoded', '材料_encoded']]  # 自变量 
# y = data['损耗']  # 因变量
# 
# # 使用 train_test_split 划分数据，80% 为训练集，20% 为测试集
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
# 
# # print(f"训练集大小: {X_train.shape}, 测试集大小: {X_test.shape}")
# 
# font_path = '/System/Library/Fonts/Hiragino Sans GB.ttc'
# my_font = fm.FontProperties(fname=font_path)
# 
# plt.rcParams['font.family'] = my_font.get_name()
# plt.rcParams['axes.unicode_minus'] = False  # 解决负号显示为方块的问题
# 
# # 初始化 XGBoost 模型
# xgboost_model = xgb.XGBRegressor()
# 
# # 训练模型
# xgboost_model.fit(X_train, y_train)
# 
# # 在测试集上预测
# y_pred = xgboost_model.predict(X_test)
# 
# # 计算均方误差
# mse = mean_squared_error(y_test, y_pred)
# print(f"XGBoost 回归的均方误差: {mse}")
# 
# # 查看特征的重要性
# xgb.plot_importance(xgboost_model)
# plt.title('特征重要性', fontproperties=my_font)
# plt.show()
# 
# importance = xgboost_model.get_booster().get_score(importance_type='gain')
# print("特征重要性 (Gain):", importance)


In [7]:
# # 生成部分依赖图，展示温度和频率对磁芯损耗的影响
# PartialDependenceDisplay.from_estimator(xgboost_model, X_train, [0, 1, 2], feature_names=['温度', '材料', '波形'])
# plt.show()